# Asynchronous computing and parallel algorithms

* Asynchronous programming using std::async and std::future
* Asynchronous programming using hpx::async and hpx::future
* Experimental support of the parallel algorithms in GCC and MSVC
* Full support of parallel algorithms in HPX

## Asynchronous programming using std::async and std::future

Documentation:
* [std::future](https://en.cppreference.com/w/cpp/thread/future) 
* [std::async](https://en.cppreference.com/w/cpp/thread/async)

Book:

* Williams, Anthony.C++ concurrency in action : practical multithreading. Shelter Island, NY:Manning, 2012.isbn: 9781933988771.


In [1]:
%%writefile example.cpp
#include <future>
#include <iostream>

int add(int a, int b)
{
    return a + b;
}

int main(void)
{
int a = 5;
int b = 5;
std::future<int> result = std::async(add,a,b);

std::cout<< result.get() << std::endl;

return EXIT_SUCCESS;
}

file written: example.cpp

### Compilation

We need to add `-phtread` for the asyncronous execution

In [2]:
%%bash
g++ example.cpp -pthread -o example

### Exectution

In [3]:
%%bash
./example

10

## Asynchronous programming using hpx::async and hpx::future


In [4]:
#include<run_hpx.cpp>
#include <hpx/future.hpp>
#include <iostream>

In [5]:
int add(int a, int b)
{
    return a + b;
}

In [6]:
run_hpx([](){

hpx::lcos::future<int> result = hpx::async(add,5,5);

std::cout << result.get() ;

});

10

(void) @0x7fbcc7c35e60


### Example Numerical integration


For this example the Taylor series for the $\sin(x)$
function is computed. The Taylor series is given by,

$$ \sin(x) \approx = \sum\limits_{n=0}^N (-1)^{n-1} \frac{x^{2n}}{(2n)!}.$$

For the concurrent computation, the interval $[0, N]$ is split in two
partitions from $[0, N/2]$ and $[(N/2)+1, N]$, and these are computed
asynchronously using `hpx::async`. Note that each asynchronous function call
returns an `hpx::future` which is needed to synchronize the collection
of the partial results. 

In [7]:
long fact(long n) {
   if (n == 0 || n == 1)
   return 1;
   else
   return n * fact(n - 1);
}



In [8]:
#include <cmath>

// Define the partial taylor function
double taylor(size_t begin, size_t end, size_t n, double x)
{
 double denom = fact(2 * n);
 double res = 0;
 for (size_t i = begin; i != end; ++i)
 {
 res += std::pow(-1, i - 1) * std::pow(x, 2 * n) / denom;
 }
 return res;
}

In [9]:
run_hpx([](){

// Compute the Talor series sin(2.0) for 25 iterations
size_t n = 25;

// Launch two concurrent computations of each partial result
hpx::future<double> f1 = hpx::async(taylor, 0, n / 2, n, 2.);
hpx::future<double> f2 = hpx::async(taylor, (n / 2) + 1, n, n, 2.);

// Introduce a barrier to gather the results
double res = f1.get() + f2.get();

// Print the result
std::cout << "Sin(2.) = " << res << std::endl;

});

Sin(2.) = -0.000691055


(void) @0x7fbcc7c35e60



![alt text](https://www.memesmonkey.com/images/memesmonkey/fd/fd382388e9c6585ee8f9c7a12533f2f6.jpeg "Title")





## Experimental support of the parallel algorithms in GCC and MSVC

* C++17 added support for parallel algorithms to thestandard library, to help programs take advantage ofparallel execution for improved performance.
* Parallelized versions of 69 algorithms from <algorithm>, <numeric> and <memory> are available
* Only recently released compilers (gcc >= 9 and MSVC >= 19.14) 1implement these new features and some of them are still experimental

Links:
* [Compiler support](https://en.cppreference.com/w/cpp/compiler_support)
* [Parallelism in C++ 17](https://en.cppreference.com/w/cpp/experimental/parallelism)



### Example 

We want to compute the sum of all elements in some `std::vector` n 

$$ \sum_{i=0}^N r+= n[i] $$

in sequential and parallel using the C++ Standard template library without using any `for` loop.

Links:

* [std::vector](https://en.cppreference.com/w/cpp/container/vector)
* [std::iterators](https://en.cppreference.com/w/cpp/iterator/iterator)
* [std::fill](https://en.cppreference.com/w/cpp/algorithm/fill)
* [std::accumulate](https://en.cppreference.com/w/cpp/algorithm/accumulate)
* [std::reduce](https://en.cppreference.com/w/cpp/algorithm/reduce)

#### Naive implementation

In [11]:
size_t len = 1000000000;
std::vector<int> n = std::vector<int>(len);

for (size_t i = 0; i < n.size(); i++)
n[i] = -1;

int result = 0;

for (size_t i = 0; i < n.size(); i++)

result += n[i];

std::cout << "Result= " << result << std::endl;


Result= -1000000000


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fbcd9436500


#### Implementation using the C++ STL sequential algorithms

In [12]:
std::vector<int> n2 = std::vector<int>(len);

std::fill(n2.begin(),n2.end(),-1);

result = std::accumulate(n2.begin(),n2.end(),0.0);

std::cout << "Result= " << result << std::endl;

Result= -1000000000


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fbcd9436500


#### Implementation using the C++ STL parallel algorithms

In [22]:
%%writefile parallel.cpp
#include<execution>
#include<iostream>

int main(void){

size_t len = 1000000000;
std::vector<int> n = std::vector<int>(len);

std::fill(n.begin(),n.end(),-1);

int result = 0;

result = std::reduce(std::execution::par,n.begin(), n.end());

std::cout << "Result= " << result << std::endl;

return EXIT_SUCCESS;
}

file written: parallel.cpp

##### Compilation

The parallism for the gcc is based-on the [Threading Building Blocks](https://github.com/oneapi-src/oneTBB) libary. Therfore, we need to add `-ltbb` to the compiler. Since these feature are experimental, we need to use the following C++ standard `-std=c++1z`.

In [23]:
%%bash 
g++ -std=c++1z -ltbb parallel.cpp -o parallel

In [24]:
%%bash
./parallel

Result= -1000000000

### Execution policies

* `std::reduce(std::execution::par,n.begin(), n.end());` - Parallel execution
* `std::reduce(std::execution::seq,n.begin(), n.end());` - Sequentiell execution
* `std::reduce(std::execution::par_unseq,n.begin(), n.end());` - Parallel execution with vectorization

[Documentation](https://en.cppreference.com/w/cpp/experimental/parallelism)

Fore more details: CppCon 2016: Bryce AdelsteinLelbach [“The C++17 Parallel Algorithms Library and Beyond”](https://www.youtube.com/watch?v=Vck6kzWjY88)